In [ ]:
# 安裝faiss
!pip install faiss-cpu --no-cache

In [ ]:
# 更新軟體包資訊
!sudo apt-get update

In [ ]:
# 安裝java
!sudo apt-get install openjdk-11-jdk

In [ ]:
# 建立反向索引
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /user_data/preprocess/wiki_process/wikijson_1024 \
  --language zh \
  --index /user_data/preprocess/wiki_index/wiki_index_1024 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

In [102]:
# 使用反向索引
from pyserini.search.lucene import LuceneSearcher
import time

searcher = LuceneSearcher('wiki_index/wiki_index_256') # 反向索引資料夾
searcher.set_language('zh')
search_key = '癌症'

start = time.time()
hits = searcher.search(search_key,  k=15) # 預設k=10
end = time.time()

print("排名\t文章編號\tBM25分數")
for i in range(len(hits)):
    print(f'{i+1}\t{hits[i].docid}\t{hits[i].score:.5f}')

print("\n執行時間：%f 秒" % (end - start))

排名	文章編號	BM25分數
1	3966268	5.69150
2	6607631	5.69150
3	6923121	5.65760
4	7430046	5.63920
5	7430046	5.63920
6	7156607	5.61580
7	1665630	5.60270
8	764174	5.60270
9	84100	5.60270
10	6607631	5.59540
11	6923121	5.59540
12	3683109	5.57520
13	3683109	5.57520
14	72879	5.57520
15	7779794	5.57520

執行時間：0.001694 秒


In [107]:
from pyserini.search.lucene import LuceneSearcher
import json

searcher = LuceneSearcher('wiki_index/wiki_index_256')
hits = searcher.search('癌症', k=15)  
doc_id = [hit.docid for hit in hits]

raw_doc = searcher.doc(doc_id[0]).raw()
content = json.loads(raw_doc)['contents']

print(content)

維生素C（），又稱抗壞血酸（），是一種存在於各種食物的維他命，也被作為營養補充品銷售，可用於預防及治療壞血病。維他命C是參與組織修復和某些神經遞質中的酶促生產。它是幾種酶的功能中所必需的，並且對於免疫系統運作很重要。它還可用作抗氧化劑。
證據不支持將其用於預防普通感冒。然而有些證據顯示，經常服用可縮短患感冒的時間。目前尚未清楚營養補充劑是否會對癌症、心血管疾病或認知障礙症有所影響。服用可透過口服或注射。
維他命C通常耐受性良好。大劑量服用可能會導致胃腸道不適、頭痛、睡眠困難及皮膚潮紅。在懷孕期間，正常劑量是


In [105]:
# 比較暴力搜尋
import os
import json
import time

search_key = '癌症'
folder_path = 'wiki_process/wikijson_1024'
list = []

start = time.time()
for filename in os.listdir(folder_path): # 找資料夾中的所有json檔
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as f:
            json_data_list = json.load(f)
            
            for json_data in json_data_list:
                if search_key in json_data.get('contents', ''):
                    list.append(json_data) 
end = time.time()

print("執行時間：%f 秒" % (end - start))


執行時間：11.485053 秒
